# Brain Tumor Classification using CNN

This notebook implements a Convolutional Neural Network (CNN) to classify brain MRI scans into two categories:
1. Brain Tumor Present (Yes)
2. Brain Tumor Absent (No)

The model will be trained on a dataset of MRI scans and will learn to identify the presence of tumors with high accuracy.

## 1. Import Required Libraries

We'll import the necessary libraries for building our CNN model:
- Keras/TensorFlow for building and training the CNN
- scikit-learn for data splitting and preprocessing
- numpy for numerical operations
- matplotlib for visualization

In [5]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import Keras modules
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.preprocessing.image import load_img, img_to_array

# Import scikit-learn modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# For reading image files
import os
import glob

## 2. Data Import and Preprocessing

In this section, we will:
1. Load MRI images from the dataset
2. Resize all images to 128x128x3 dimension
3. Convert labels to one-hot encoded format
4. Combine data from both classes (tumor present/absent)

The dataset contains two folders:
- 'yes': Contains MRI scans with brain tumors
- 'no': Contains MRI scans without brain tumors

In [ ]:
# This cell has been updated with improved image loading code

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

In [7]:
# Initialize lists to store data
data = []  # For storing image data
result = []  # For storing labels
paths = []  # For storing image paths

# Image dimensions
IMG_HEIGHT = 128
IMG_WIDTH = 128
CHANNELS = 3

# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    try:
        # Load and resize image
        img = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
        # Convert to array and normalize
        img_array = img_to_array(img) / 255.0
        return img_array
    except Exception as e:
        print(f"Error loading image {image_path}: {str(e)}")
        return None

# Function to get all image files with various extensions
def get_image_files(directory):
    extensions = ['*.jpg', '*.jpeg', '*.JPG', '*.JPEG', '*.png', '*.PNG']
    image_files = []
    for ext in extensions:
        image_files.extend(glob.glob(os.path.join(directory, ext)))
    return image_files

print("Loading and preprocessing images...")

# Load 'yes' (tumor) images
yes_path = 'brain_tumor_dataset/yes'
yes_files = get_image_files(yes_path)
print(f"Found {len(yes_files)} images in tumor class")

for img_path in yes_files:
    img = load_and_preprocess_image(img_path)
    if img is not None:
        data.append(img)
        result.append(1)  # 1 for tumor present
        paths.append(img_path)

# Load 'no' (no tumor) images
no_path = 'brain_tumor_dataset/no'
no_files = get_image_files(no_path)
print(f"Found {len(no_files)} images in no-tumor class")

for img_path in no_files:
    img = load_and_preprocess_image(img_path)
    if img is not None:
        data.append(img)
        result.append(0)  # 0 for tumor absent
        paths.append(img_path)

# Convert lists to numpy arrays
data = np.array(data)
result = np.array(result)

# One-hot encode the labels
encoder = OneHotEncoder(sparse=False)
result = encoder.fit_transform(result.reshape(-1, 1))

print("\nDataset Summary:")
print("Total images loaded:", len(data))
print("Dataset shape:", data.shape)
print("Labels shape:", result.shape)
print(f"Number of tumor images: {sum(result[:,1])}")
print(f"Number of no-tumor images: {sum(result[:,0])}")

Loading and preprocessing images...
Found 310 images in tumor class
Found 196 images in no-tumor class


TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

## 3. Train-Test Split

We'll split our dataset into:
- Training set (80% of data)
- Testing set (20% of data)

This split ensures we can evaluate our model on unseen data.

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    data, 
    result,
    test_size=0.2,    # 20% for testing
    random_state=42    # For reproducibility
)

# Print the shapes of training and testing sets
print("Training set shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Testing set shape:", X_test.shape)
print("Testing labels shape:", y_test.shape)

## 4. CNN Model Architecture

We'll build a CNN with the following components:
1. Convolutional layers with ReLU activation
2. MaxPooling layers for downsampling
3. Batch Normalization for stabilizing training
4. Dropout layers to prevent overfitting
5. Dense layers for final classification

The architecture follows this pattern:
- Conv2D -> ReLU -> BatchNorm -> MaxPool -> Dropout
- Conv2D -> ReLU -> BatchNorm -> MaxPool -> Dropout
- Flatten
- Dense -> Softmax

In [ ]:
# Initialize the CNN model
model = Sequential()

# First set of Conv -> ReLU -> BatchNorm -> MaxPool -> Dropout
model.add(Conv2D(32, (2, 2), input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS)))
model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Second set of Conv -> ReLU -> BatchNorm -> MaxPool -> Dropout
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten layer
model.add(Flatten())

# Dense layers with dropout
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # 2 classes: tumor present/absent

# Compile the model
model.compile(
    optimizer='adamax',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()

## 5. Model Training

We'll train the model for 30 epochs with the following parameters:
- Batch size: 32
- Validation split: 20% of training data
- Monitor both training and validation loss/accuracy

In [ ]:
# Train the model
history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=30,
    validation_split=0.2,
    verbose=1
)

# Get the training and validation metrics
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Print final metrics
print("\nFinal Training Accuracy:", train_accuracy[-1])
print("Final Validation Accuracy:", val_accuracy[-1])
print("Final Training Loss:", train_loss[-1])
print("Final Validation Loss:", val_loss[-1])

## 6. Model Evaluation and Visualization

Let's visualize the training process by plotting:
1. Training vs Validation Loss
2. Training vs Validation Accuracy

This will help us understand if the model is overfitting or underfitting.

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

# Plot training & validation loss
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training & validation accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("\nTest Accuracy:", test_accuracy)
print("Test Loss:", test_loss)

## 7. Testing Model Predictions

Let's test our model on some sample images from the test set and visualize:
1. The input MRI scan
2. The true label
3. The model's prediction with confidence score

In [ ]:
# Function to make predictions
def predict_tumor(image, true_label):
    # Make prediction
    pred = model.predict(image.reshape(1, IMG_HEIGHT, IMG_WIDTH, CHANNELS))
    pred_class = np.argmax(pred)
    pred_prob = pred[0][pred_class]
    
    # Convert true label from one-hot encoding
    true_class = np.argmax(true_label)
    
    # Create labels for display
    class_labels = ['No Tumor', 'Tumor Present']
    
    # Plot the image and predictions
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.title(f'True: {class_labels[true_class]}\nPrediction: {class_labels[pred_class]} ({pred_prob:.2%})')
    plt.axis('off')
    plt.show()

# Test on a few random images from test set
for i in range(3):
    idx = np.random.randint(0, len(X_test))
    predict_tumor(X_test[idx], y_test[idx])